In [ ]:
import praw
import pandas as pd
import datetime
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
import twint
import pickle
import nltk
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import datetime


In [ ]:
def gen_time_window(start, end, step, length):
    
    start_datetime_object = datetime.datetime.strptime(start, '%Y-%m-%d')
    
    end_datetime_object = datetime.datetime.strptime(end, '%Y-%m-%d')

    iterator = start_datetime_object
    output = []
    while iterator < end_datetime_object:
        since = str(iterator)[:10]
#         until = str(iterator + datetime.timedelta(days=step))[:10]
        until = str(min(end_datetime_object, iterator + datetime.timedelta(days=length)))[:10]
        output.append((since,until))
        iterator += datetime.timedelta(days=step)
    
    return output

In [ ]:
gen_time_window('2018-01-01', '2020-06-04', 14, 1)

In [ ]:
def get_tweets(search, since, until, limit=100, db=True):
    dbname = search.replace(" ","_") + ".db"
    c = twint.Config()
    c.Search = search
    c.Limit = limit # If not specified, scrapes all...
    c.Pandas = True
    c.Pandas_clean = True
    c.Since = since
    c.Until = until
    c.Database = dbname
    c.Hide_output = True
    
    twint.run.Search(c)
    if db==False:
        return twint.output.panda.Tweets_df
    else: print("Done: "+ since + " to " + until, twint.output.panda.Tweets_df.shape[0], "entries")

In [ ]:
def scrape_to_db(keyword, start_date, end_date, step, length, limit):
    dates = gen_time_window(start_date, end_date, step, length)
    for i in dates:
        get_tweets(keyword, i[0], i[1], limit=limit)

In [ ]:
scrape_to_db(keyword = "spacex rocket", 
             start_date = '2020-01-01', 
             end_date = '2020-06-01', 
             step = 30, 
             length = 1, 
             limit = 100)

In [ ]:
scrape_to_db(keyword = "digital health", 
             start_date = '2018-01-01', 
             end_date = '2020-06-01', 
             step = 14, 
             length = 1, 
             limit = 1000)

In [ ]:
tweets = get_tweets("digital health", '2020-06-03','2020-06-04', limit=100)
tweets = get_tweets("digital health", '2020-06-01','2020-06-02', limit=100)

# tweets.shape

In [ ]:
# pd.set_option("max_columns", None)

# pd.reset_option("max_rows")


In [ ]:
tweets

In [ ]:
tweets['tweet']

In [ ]:
import json
with open('gender/top_words.json') as json_file:
    top_words = json.load(json_file)

In [ ]:
def find_features(top_words, text):
    feature = {}
    for word in top_words:
        feature[word] = word in text.lower()
    return feature

In [ ]:
NB_classifier = pickle.load(open('gender/gender_NB_classifier.sav', 'rb'))

In [ ]:
tweets['tweet'][0]

In [ ]:
NB_classifier.classify(find_features(top_words, tweets['tweet'][0]))

In [ ]:
tweets['gender'] = tweets['tweet'].apply(lambda x: NB_classifier.classify(find_features(top_words, x)))

In [ ]:
tweets['gender'].value_counts()

In [1]:
!jupyter nbconvert --to script gender_classif.ipynb

[NbConvertApp] Converting notebook gender_classif.ipynb to script
[NbConvertApp] Writing 3194 bytes to gender_classif.py
